In [1]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{"host": "localhost", "port": 9200}],
    http_compress=True,
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

response = client.cluster.health()
print(response)

{'cluster_name': 'docker-cluster', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'discovered_master': True, 'discovered_cluster_manager': True, 'active_primary_shards': 6, 'active_shards': 6, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [33]:
client.indices.get_alias("*")

{'.opensearch-observability': {'aliases': {}},
 '.plugins-ml-config': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}},
 '.opensearch-sap-log-types-config': {'aliases': {}},
 'reports_en': {'aliases': {}},
 'reports_de': {'aliases': {}}}

In [34]:
from datetime import datetime

client.create(
    index="reports_de",
    id="xxx",
    body={
        "document_id": "kldfjlk",
        "language": "en",
        "groups": [1, 2, 3],
        "pacs_aet": "foobar",
        "pacs_name": "Foobar",
        "pacs_link": "http://xxx.foobar",
        "patient_birth_date": datetime(1976, 1, 2),
        "patient_age": 33,
        "patient_sex": "M",
        "study_datetime": datetime(2020, 3, 9),
        "modalities": ["CT", "MR", "PET"],
        "body": "Dies ist ein radiologischer Befund über Knie Schmerzen.",
    },
)

{'_index': 'reports_de',
 '_id': 'xxx',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 1000,
 '_primary_term': 1}

In [35]:
client.count(index="reports_de")

{'count': 1000,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [36]:
client.search(index="reports_de")

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1000, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'reports_de',
    '_id': 'bef3f74d-5ca9-457c-8d5f-2d36678075a7',
    '_score': 1.0,
    '_source': {'document_id': 'bef3f74d-5ca9-457c-8d5f-2d36678075a7',
     'language': 'de',
     'groups': [1],
     'pacs_aet': 'one',
     'pacs_name': 'particularly',
     'pacs_link': 'http://www.knight-moore.com/',
     'patient_birth_date': '1995-06-18',
     'patient_age': 22,
     'patient_sex': 'M',
     'study_description': 'Floor always chair high. Try role campaign.',
     'study_datetime': '2018-06-05T08:12:24.802429+00:00',
     'modalities': ['CT', 'MR', 'US', 'DX'],
     'body': 'Befund:\n\nRöntgen Thorax in zwei Ebenen zeigt eine pulmonale Infiltration im rechten mittleren Lungenfeld, die radiologisch mit einer Pneumonie vereinbar ist. Keine Zeichen einer Pleuraerguss oder Pneumothorax. Ke

In [37]:
client.search(
    index="reports_de",
    body={
        "query": {
            "bool": {
                "must": {
                    "simple_query_string": {"query": "pneumothorax"},
                },
                "filter": [{"terms": {"modalities": ["CT"]}}, {"term": {"patient_sex": "M"}}],
            },
        },
    },
)

{'took': 60,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 67, 'relation': 'eq'},
  'max_score': 1.6169636,
  'hits': [{'_index': 'reports_de',
    '_id': 'e02f7a5b-ce1a-4950-a45a-8ca2c54ffcd5',
    '_score': 1.6169636,
    '_source': {'document_id': 'e02f7a5b-ce1a-4950-a45a-8ca2c54ffcd5',
     'language': 'de',
     'groups': [1],
     'pacs_aet': 'set',
     'pacs_name': 'impact',
     'pacs_link': 'http://johnson-taylor.com/',
     'patient_birth_date': '1911-01-01',
     'patient_age': 112,
     'patient_sex': 'M',
     'study_description': 'Action everybody find room try age yard.',
     'study_datetime': '2023-09-20T23:52:48.247572+00:00',
     'modalities': ['CT', 'MR'],
     'body': 'Befund:\n\nIm Röntgen des Thorax zeigt sich eine leichte Vergrößerung des rechten Herzens, ohne Anzeichen einer akuten Herzinsuffizienz. Die Lungenfelder sind frei von Infiltraten und Pneumoniezeichen. Keine Anzeichen einer P

In [4]:
client.search(index="reports_de", body={"size": 10, "query": {"term": {"groups": 1}}})

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1000, 'relation': 'eq'},
  'max_score': 0.0004996253,
  'hits': [{'_index': 'reports_de',
    '_id': 'b42c20f3-7367-4ca1-8bea-3811c1bd584b',
    '_score': 0.0004996253,
    '_source': {'document_id': 'b42c20f3-7367-4ca1-8bea-3811c1bd584b',
     'language': 'de',
     'groups': [1],
     'pacs_aet': 'career',
     'pacs_name': 'professional',
     'pacs_link': 'http://www.garcia-singleton.com/',
     'patient_birth_date': '1914-07-09',
     'patient_age': 102,
     'patient_sex': 'M',
     'study_description': 'Wonder time tree. Police rate technology baby me mission.',
     'study_datetime': '2016-10-31T03:09:47.948128+00:00',
     'modalities': ['DX', 'PT', 'US', 'CT', 'MR'],
     'body': 'Befund:\n\nRöntgen Thorax in zwei Ebenen zeigt eine pulmonale Infiltration im rechten mittleren Lungenfeld, die radiologisch mit einer Pneumonie vereinbar ist. Keine Ze

In [39]:
response = client.search(
    index="reports_de",
    body={"size": 10, "query": {"range": {"patient_birth_date": {"gte": datetime(1910, 1, 1)}}}},
)

bd = response["hits"]["hits"][0]["_source"]["patient_birth_date"]
datetime.fromisoformat(bd)

datetime.datetime(1995, 6, 18, 0, 0)

In [2]:
response = client.search(
    index="reports_de",
    body={
        # "_source": False,
        "fields": ["patient_sex"],
        "query": {"query_string": {"query": "Schmerzen rechten Knie"}},
        "highlight": {
            # "fragment_size": 10,
            "fields": {"body": {}},
        },
    },
)

response

{'took': 146,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 557, 'relation': 'eq'},
  'max_score': 7.028917,
  'hits': [{'_index': 'reports_de',
    '_id': 'a9975d11-77ed-4f6a-8c20-85fa97a35f50',
    '_score': 7.028917,
    '_source': {'document_id': 'a9975d11-77ed-4f6a-8c20-85fa97a35f50',
     'language': 'de',
     'groups': [1],
     'pacs_aet': 'little',
     'pacs_name': 'spend',
     'pacs_link': 'https://davis.info/',
     'patient_birth_date': '1955-10-03',
     'patient_age': 67,
     'patient_sex': 'M',
     'study_description': 'Understand local us. Speech total choose travel claim.',
     'study_datetime': '2022-12-28T03:56:28.988562+00:00',
     'modalities': ['DX', 'US', 'CT'],
     'body': 'Bildgebender Befund:\n\nUntersuchung: MRT des rechten Knies\nIndikation: Schmerzen und Schwellung im rechten Knie\n\nBefund:\n- Intakte Knochenstruktur ohne Frakturen oder Dislokationen.\n- In der Gelenkhöhle fi